## Communication neuroscience on a shoestring

### Compliance Analysis

This script performs preprocessing steps as in the first notebook, but separates the data out according to sessions (1 and 2) and compliance-gaining condition (experimental group: 'request' vs. control group: 'control').

#### Import modules

In [1]:
import os, pickle, warnings, scipy.stats, mne, collections
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
from utils import utils
from autoreject import get_rejection_threshold, AutoReject, compute_thresholds 

#### Load data into MNE objects

In [2]:
subjects             = list(np.load('../data/subjects.npy', allow_pickle=True))
compliance_condition = list(np.load('../data/compliance_condition.npy', allow_pickle=True))

sessions_to_import   = [1, 2]
results_drop_run     = []

for current_session in sessions_to_import:
    print('Current Session')
    print(current_session)
    print('--------------')

    stats_data = np.zeros(4)

    all_evokeds = []
    trials_res = []

    for curr_subject in subjects[:]:
        print("Current Subject #" + str(curr_subject))
        
        raw = utils.load_data('../data/Study_AB/', 
                              subject_nb = curr_subject, sfreq = 256., 
                              session_nb = current_session);

        raw.filter(0.1, 15, method='iir', verbose= False);

        events = mne.find_events(raw, min_duration=0, shortest_event = 0);
        event_id = {'Positive': 1, 'Neutral': 2};

        # Creating an Epoch object with standard parameters 
        epochs = mne.Epochs(raw, events = events, 
                        event_id = event_id, 
                        tmin = -0.1, tmax = 0.8, baseline = (-0.1,0),
                        detrend = 1, preload = True, verbose = False,
                        picks=[0,3]); 

        #Getting the automated rejection threshold for this file - otherwise same parameters as above
        print('------------ Autoreject: Get Threshold ------------')
        reject = get_rejection_threshold(epochs, decim=2)

        epochs = mne.Epochs(raw, events=events, 
                        event_id=event_id, 
                        tmin = -0.1, tmax = 0.8, baseline = (None, 0),
                        reject = reject, detrend = 1,preload=True,verbose=False,
                        picks=[0,3]);

        epochs.drop_bad(reject=reject)

        ar = AutoReject( n_interpolate= [0], random_state=42, n_jobs=1, verbose=False );
        epochs_ar, reject_log = ar.fit_transform(epochs, return_log=True)
        stats_data[1] = np.round(epochs_ar['Neutral'].get_data().shape[0])
        stats_data[2] = np.round(epochs_ar['Positive'].get_data().shape[0])

        epochs_ar.drop_bad()
        sample_drop = (1 - len(epochs_ar.events)/len(events)) * 100
        stats_data[3] = sample_drop
        trials_res.append(stats_data.copy())

    results_drop_run.append(np.asarray(trials_res)[:,3])

Current Session
1
--------------
Current Subject #A-3
Creating RawArray with float64 data, n_channels=5, n_times=30564
    Range : 0 ... 30563 =      0.000 ...   119.387 secs
Ready.
107 events found
Event IDs: [1 2]
------------ Autoreject: Get Threshold ------------
Estimating rejection dictionary for eeg
Current Subject #A-5
Creating RawArray with float64 data, n_channels=5, n_times=30564
    Range : 0 ... 30563 =      0.000 ...   119.387 secs
Ready.
105 events found
Event IDs: [1 2]
------------ Autoreject: Get Threshold ------------
Estimating rejection dictionary for eeg
Current Subject #A-6
Creating RawArray with float64 data, n_channels=5, n_times=30564
    Range : 0 ... 30563 =      0.000 ...   119.387 secs
Ready.
107 events found
Event IDs: [1 2]
------------ Autoreject: Get Threshold ------------
Estimating rejection dictionary for eeg
Current Subject #A-7
Creating RawArray with float64 data, n_channels=5, n_times=30552
    Range : 0 ... 30551 =      0.000 ...   119.340 secs


Event IDs: [1 2]
------------ Autoreject: Get Threshold ------------
Estimating rejection dictionary for eeg
Current Subject #B1-8
Creating RawArray with float64 data, n_channels=5, n_times=38100
    Range : 0 ... 38099 =      0.000 ...   148.824 secs
Ready.
120 events found
Event IDs: [1 2]
------------ Autoreject: Get Threshold ------------
Estimating rejection dictionary for eeg
Current Subject #B1-9
Creating RawArray with float64 data, n_channels=5, n_times=38088
    Range : 0 ... 38087 =      0.000 ...   148.777 secs
Ready.
120 events found
Event IDs: [1 2]
------------ Autoreject: Get Threshold ------------
Estimating rejection dictionary for eeg
Current Subject #B1-14
Creating RawArray with float64 data, n_channels=5, n_times=38256
    Range : 0 ... 38255 =      0.000 ...   149.434 secs
Ready.
120 events found
Event IDs: [1 2]
------------ Autoreject: Get Threshold ------------
Estimating rejection dictionary for eeg
Current Subject #B1-15
Creating RawArray with float64 data, n_

KeyboardInterrupt: 

In [ ]:
drop_rate_df              = pd.DataFrame(np.asarray(results_drop_run).T, columns = ['session1', 'session2'], index = subjects)
drop_rate_df['condition'] = compliance_condition
drop_rate_df.to_csv('../data/drop_rate_results.csv')
drop_rate_df.head()

In [ ]:
df = drop_rate_df.groupby(['condition']).mean().T
ax = df.plot(kind='bar', colors = ['lightgray', 'black']);
ax.legend(loc='lower left', frameon=True);

In [ ]:
drop_rate_df

In [ ]:
drop_rate_df.groupby(['condition']).mean()

In [ ]:
drop_rate_df.groupby(['condition']).std()

In [ ]:
scipy.stats.ttest_ind( drop_rate_df['session1'][drop_rate_df['condition']=='control'],
                       drop_rate_df['session1'][drop_rate_df['condition']=='request'])

In [ ]:
scipy.stats.ttest_ind( drop_rate_df['session2'][drop_rate_df['condition']=='control'],
                       drop_rate_df['session2'][drop_rate_df['condition']=='request'])

In [ ]:
scipy.stats.ttest_rel( drop_rate_df['session1'][drop_rate_df['condition']=='control'],
                       drop_rate_df['session2'][drop_rate_df['condition']=='control'])

In [ ]:
scipy.stats.ttest_rel( drop_rate_df['session1'][drop_rate_df['condition']=='request'],
                       drop_rate_df['session2'][drop_rate_df['condition']=='request'])